In [1]:
!pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [49]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time

# 웹드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.bigkinds.or.kr/v2/news/recentNews.do')
time.sleep(5)  # 페이지 로드를 기다림

# 데이터를 저장할 리스트 생성
news_data = []

current_page = 1
max_page = 1  # 원하는 페이지 수 지정

while current_page <= max_page:
    news_blocks = driver.find_elements(By.CSS_SELECTOR, 'div.news-inner')
    for block in news_blocks:
        attempts = 0
        while attempts < 3:
            try:
                # 신문사 정보 추출
                publisher = block.find_element(By.CSS_SELECTOR, 'a').text

                # 뉴스 URL과 제목 추출
                title_element = block.find_element(By.CSS_SELECTOR, 'a')
                news_url = title_element.get_attribute('href')
                title_element.click()  # 기사 제목 클릭
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.news-view-body')))
                
                # 데이터 추출
                news_title = driver.find_elements(By.CSS_SELECTOR, 'div.news-view-head > title')
                date_and_reporter = driver.find_elements(By.CSS_SELECTOR, 'ul.info > li')
                news_date = date_and_reporter[0].text if len(date_and_reporter) > 0 else '날짜 정보 없음'
                reporter_name = date_and_reporter[1].text if len(date_and_reporter) > 1 else '기자 정보 없음'
                news_content = driver.find_element(By.CSS_SELECTOR, 'div.news-view-body').text
                # 이미지 추출
                try:
                    image_element = driver.find_element(By.CSS_SELECTOR, 'div.news-view-body img')
                    image_src = image_element.get_attribute('src')
                except NoSuchElementException:
                    image_src = '이미지 없음'

                print(news_title)
                #print(image_src)

                # 데이터 리스트에 추가
                news_data.append({
                    'Date': news_date,
                    'Title': news_title,
                    'Press': publisher,
                    'Author': reporter_name,
                    'Content': news_content,
                    'Image': image_src,
                    'URL': news_url,
                })

                # 기사 상세보기 창 닫기
                # driver.find_element(By.CSS_SELECTOR, 'button.modal-close').click()
                # print(driver.find_element(By.CSS_SELECTOR, 'div.modal-footer > button').text)
                driver.find_element(By.CSS_SELECTOR, 'div.modal-footer > button').click()
                # driver.find_element(By.CSS_SELECTOR, 'button.modal-close').click()
                time.sleep(2)
                break
            except Exception as e:
                print(f"Error collecting data: {e}. Retrying...")
                time.sleep(2)
                attempts += 1

    # 다음 페이지로 이동
    if current_page < max_page:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, f'a.page-link[data-page="{current_page + 1}"]'))
        )
        next_page_button.click()
        time.sleep(5)
    current_page += 1

driver.quit()

# DataFrame 생성
df = pd.DataFrame(news_data)
df.to_csv('news_data.csv', index=False, encoding='utf-8-sig')

print("CSV 파일로 저장되었습니다.")


Error collecting data: type object 'By' has no attribute 'CSS_SELECTOTR'. Retrying...
Error collecting data: Message: element click intercepted: Element <a href="#modal-news1" data-toggle="modal" class="thumb news-detail" data-newsid="01600901.20241017170925005" style="background-image: url('/assets/v3/img/content/img-noImg.png');" title="팝업창이 열림"></a> is not clickable at point (225, 602). Other element would receive the click: <div class="modal-content">...</div>
  (Session info: chrome=130.0.6723.59)
Stacktrace:
0   chromedriver                        0x0000000100d339d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x0000000100d2c234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x000000010079810c cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001007dfa74 cxxbridge1$string$len + 381640
4   chromedriver                        0x00000001007de0dc cxxbridge1$string$len + 375088
5   chromedriver                   

In [45]:
df = pd.read_csv('news_data.csv')

df.head()

,Date,Title,Press,Author,Content,Image,URL
0,2024-10-17,NaN,NaN,김현희 기자,배우 한지현. ⓒ이혜영 기자 lhy@hankooki.com\n[스포츠한국 김현희 기...,https://www.bigkinds.or.kr/resources/images/10...,https://www.bigkinds.or.kr/v2/news/recentNews....
1,2024-10-17,NaN,NaN,최호,17일 울산시 중구 한국석유공사에서 열린 국회 산업통상자원중소벤처기업위원회의 한국석...,https://www.bigkinds.or.kr/resources/images/07...,https://www.bigkinds.or.kr/v2/news/recentNews....
2,2024-10-17,NaN,NaN,김재민,한화솔루션 큐셀부문(한화큐셀) 진천공장 내 유휴부지에 설치된 루프탑 태양광 발전. ...,https://www.bigkinds.or.kr/resources/images/04...,https://www.bigkinds.or.kr/v2/news/recentNews....
3,2024-10-17,NaN,NaN,최빛나,"티메프 사태로 온라인 e커머스에 대한 불신이 커지고 있는 가운데, 국내 온라인 채널...",https://www.bigkinds.or.kr/resources/images/02...,https://www.bigkinds.or.kr/v2/news/recentNews....
4,2024-10-17,NaN,NaN,홍예정기자 hyj27@kwnews.co.kr,◇2024 도그페스타\n춘천 강아지숲이 19일부터 20일까지 이틀간 ‘2024 도그...,https://www.bigkinds.or.kr/resources/images/01...,https://www.bigkinds.or.kr/v2/news/recentNews....
